# Machine Epsilon (Floating-Point Precision Limit)
Machine epsilon measures the smallest relative spacing of floating-point numbers near 1.0.  
It explains the precision limits of `float32` and `float64`, and why some expressions (like $1-\cos(x)$ from notebook `00`) collapse to zero when the true result is smaller than the representable spacing.

In [64]:
import numpy as np

In [65]:
DTYPES = [np.float32, np.float64]

## What is Machine Epsilon
Floating-point numbers have finite precision. Around the number 1.0, there is a *gap* between representable numbers.  

Machine epsilon ($\varepsilon$) is commonly defined as the smallest number such that:  

$1 + \varepsilon \neq 1$  

In other words, it is the smallest relative increment that changes $1.0$ in that floating-point format.

## Compute Epsilon by Halving

Algorithm:  
1. Start $eps = 1$
2. Keep halving until $1 + eps == 1$
3. The previous eps is the machine epsilon

In [66]:
def compute_machine_epsilon(dtype):
    one = dtype(1.0)
    eps = dtype(1.0)
    prev = np.inf
    
    while one + eps != one:
        prev = eps
        eps = dtype(eps / dtype(2.0))
    
    return prev

In [67]:
for dtype in DTYPES:
    eps = compute_machine_epsilon(dtype)
    print(dtype.__name__, "Machine Epsilon:", eps, " | ",format(float(eps), ".20f"))

float32 Machine Epsilon: 1.1920929e-07  |  0.00000011920928955078
float64 Machine Epsilon: 2.220446049250313e-16  |  0.00000000000000022204


## Why Epsilon Matters
Near $1.0$, floating-point cannot represent arbitrarily small changes.  
If a computation produces a result smaller than the spacing near $1$, the result may be rounded away.  

For example, if `float64` spacing near 1 is about $2 \times 10^{-16}$, then any value of the form:  

$1 + \delta \quad \text{with} \quad |\delta| < 10^{-16}$ 

may round to exactly $1.0$.  

This explains catastrophic cancellation in expressions like $1 - \cos(x)$:  
when $\cos(x)$ is rounded to $1.0$, subtraction produces 0.0 even though the true answer is nonzero.  

## Demonstration: Smallest $x$ where $1+x \neq 1$

In [68]:
def smallest_increment_near_one(dtype):
    one = dtype(1.0)
    x = dtype(1.0)
    prev = np.inf
    
    while one + x != one:
        prev = x
        x = dtype(x / dtype(2.0))
    
    return prev

In [69]:
for dtype in DTYPES:
    smallest_inc = smallest_increment_near_one(dtype)
    print(dtype.__name__, "Smallest Increment:", smallest_inc, " | ",format(float(smallest_inc), ".20f"))

float32 Smallest Increment: 1.1920929e-07  |  0.00000011920928955078
float64 Smallest Increment: 2.220446049250313e-16  |  0.00000000000000022204


## Takeaway
Machine epsilon quantifies the precision limit of a floating-point format near 1.0.  
- `float32` epsilon is about $10^{-7}$ ($\approx 7$ correct decimal digits)
- `float64` epsilon is about $10^{-16}$ ($\approx 16$ correct decimal digits)

If a computation depends on differences smaller than these spacings, floating-point rounding can erase the information entirely. This is why numerically stable reformulations matter.